# Read files summarising field work and update database
These Excel workbooks were imported on February 2022.

The scripts documented here have been created to:

- Read data from spreadsheets with field-work data
- Create records for data import into the database
- Insert or update records in the database


## Set-up
Load libraries 

In [1]:
import openpyxl
from pathlib import Path
import os
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs
import re
#import postgis

Define path to workbooks

In [2]:
repodir = Path("../../") 
inputdir = repodir / "data" / "field-form"

 ### DB connection parameters and helper functions
 
 Database credentials are stored in a database.ini file

In [3]:
filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

Define a function to batch process insert or update queries:

In [4]:
def batch_upsert(params,table,records,keycol,idx, execute=False,useconn=None):
    if useconn is None:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
    else:
        conn = useconn
    cur = conn.cursor()
    #postgis.register(cur)
    updated_rows=0

    for record in records:
        if len(record.keys())>len(keycol):
            if 'geom' in record.keys():
                the_geom=record['geom']
                record['geom']='GEOMSTR'
            if idx is not None:
                qrystr = "INSERT INTO %s (%s) values %s ON CONFLICT ON CONSTRAINT %s DO UPDATE SET %s"
                upd=list()
                for k in record.keys():
                    if k not in keycol:
                        upd.append("{col}=EXCLUDED.{col}".format(col=k))
                qry = cur.mogrify(qrystr, (AsIs(table),
                                AsIs(','.join(record.keys())),
                                tuple(record.values()),
                                AsIs(idx),
                                AsIs(','.join(upd))
                               ))
            else:
                qrystr = "INSERT INTO %s (%s) values %s ON CONFLICT DO NOTHING"
                qry = cur.mogrify(qrystr, (AsIs(table),
                                AsIs(','.join(record.keys())),
                                tuple(record.values())
                               ))

            if 'geom' in record.keys():
                qry=qry.decode('utf-8')
                qry=qry.replace("'GEOMSTR'",the_geom)
                record['geom']=the_geom

            if execute:
                cur.execute(qry)
                if cur.rowcount > 0:
                    updated_rows = updated_rows + cur.rowcount
            else:
                print(qry)
            
    conn.commit()        
    cur.close()
    print("%s rows updated" % (updated_rows))
        
    if useconn is None and conn is not None:
        conn.close()
        print('Database connection closed.')

        

Just a test with random data, use `execute=False` to print the query:

In [5]:
record={'site_label':'test','geom':"ST_GeomFromText('POINT(1 2)', 4326)"}
batch_upsert(params,"form.field_site",(record,),keycol=('site_label',), idx='field_site_pkey1',execute=False)

Connecting to the PostgreSQL database...
INSERT INTO form.field_site (site_label,geom) values ('test', ST_GeomFromText('POINT(1 2)', 4326)) ON CONFLICT ON CONSTRAINT field_site_pkey1 DO UPDATE SET geom=EXCLUDED.geom
0 rows updated
Database connection closed.


In [6]:
batch_upsert(params,"form.field_site",(record,),keycol=('site_label',), idx=None,execute=False)

Connecting to the PostgreSQL database...
INSERT INTO form.field_site (site_label,geom) values ('test', ST_GeomFromText('POINT(1 2)', 4326)) ON CONFLICT DO NOTHING
0 rows updated
Database connection closed.


## Read workbooks
Each spreadsheet has a slightly different structure, so these scripts have to be adapted for each case.

### List of workbooks/spreadsheets in directory

In [7]:
os.listdir(inputdir)

['UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx',
 '.DS_Store',
 'UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx',
 '~$Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm',
 'SthnNSWRF_data_bionet2.xlsx',
 'UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx',
 'UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx',
 '~$UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx',
 'UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx',
 'RobertsonRF_data_bionet2.xlsx',
 'Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm']

In [8]:
valid_files = ['SthnNSWRF_data_bionet2.xlsx',
               'UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx',
               'UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx',
               'UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx',
               'UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx',
               'UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx',
               'RobertsonRF_data_bionet2.xlsx',
               'Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm']

Here we create an index of worksheets and column headers for each file

In [9]:
wbindex=dict()
for workbook_name in valid_files:
    inputfile=inputdir / workbook_name
    # using data_only=True to get the calculated cell values
    wb = openpyxl.load_workbook(inputfile,data_only=True)
    wbindex[workbook_name]=dict()
    for ws in wb.worksheets:
        wbindex[workbook_name][ws._WorkbookChild__title]=[list(),list()]
        for k in range(1,ws.max_column):
            wbindex[workbook_name][ws._WorkbookChild__title][0].append(ws.cell(row=1,column=k).value)
            wbindex[workbook_name][ws._WorkbookChild__title][1].append(ws.cell(row=2,column=k).value)
        

In [10]:
wbindex.keys()

dict_keys(['SthnNSWRF_data_bionet2.xlsx', 'UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx', 'UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx', 'UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx', 'UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx', 'UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx', 'RobertsonRF_data_bionet2.xlsx', 'Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm'])

In [11]:
wbindex['SthnNSWRF_data_bionet2.xlsx'].keys()

dict_keys(['Site', 'Fire', 'Structure', 'Floristics', 'Reference', 'Info', 'Sheet1'])

In [12]:
wbindex['SthnNSWRF_data_bionet2.xlsx']['Fire'][0][0:11]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3']

In [13]:
filename='SthnNSWRF_data_bionet2.xlsx'
worksheet='Fire'
wb = openpyxl.load_workbook(inputdir / filename,data_only=True)
ws = wb[worksheet]

In [14]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records=list()
for sw in col_dicts:
    for j in range(2,ws.max_row+1):
        item = ws[j]
        record=dict()
        comms=list()
        for k in sw.keys():
            vals=item[sw[k]].value
            if vals is not None:
                if k == 'fire_date':
                    if isinstance(vals,datetime):
                        record['fire_date']=str(vals.date())
                        record['earliest_date']=vals.date()
                        record['latest_date']=vals.date()
                    else:
                        record['fire_date']=str(vals)
                        comms.append('Fire date given as: %s' % vals)
                else:
                    record[k]=vals
        if len(comms)>0:
            record['notes'] = comms
        if len(record)>1:
            records.append(record)


In [15]:
records[10]

{'site_label': 'DeuaRF',
 'fire_date': '>1975-76',
 'how_inferred': 'NPWS fire records prescribed burn but unlikely to have reached site',
 'cause_of_ignition': 'prescribed',
 'notes': ['Fire date given as: >1975-76']}

In [16]:
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx=None,execute=True)

Connecting to the PostgreSQL database...
0 rows updated
Database connection closed.


#### Create fire history records

This is a lower level function that will create a field sample record from an `item` (a row in the spreadsheet), using the dictionary or "switch" in `col_dicts`:

In [106]:
def create_fire_history_record(item,col_dicts):
    records=list()
    for sw in col_dicts:
        record=dict()
        comms=list()
        if item[sw['site_label']].value == 'Site':
            continue
        for k in sw.keys():
            vals=item[sw[k]].value
            if vals is not None:
                if k == 'fire_date':
                    if isinstance(vals,datetime):
                        record['fire_date']=str(vals.date())
                        record['earliest_date']=vals.date()
                        record['latest_date']=vals.date()
                    elif isinstance(vals,int):
                        record['fire_date']=str(vals)
                        if vals>0:
                            record['earliest_date']=datetime(vals,1,1).date()
                            record['latest_date']=datetime(vals,12,31).date()
                        else:
                            comms.append('Fire date is missing or empty')
                    elif vals.isnumeric():
                        record['fire_date']=str(vals)
                        record['earliest_date']=datetime(int(vals),1,1).date()
                        record['latest_date']=datetime(int(vals),12,31).date()
                    else:
                        record['fire_date']=vals
                        comms.append('Fire date given as: %s' % vals)
                        found=re.findall("[<>]",vals)
                        for i in found:
                            comms.append('max/min value given')
                            vals=vals.replace(i,"")
                        ws=vals.split("-")
                        if len(ws)==2:
                            if len(ws[0])==4 and ws[0].isnumeric():
                                record['earliest_date']=datetime(int(ws[0]),1,1).date()
                            if len(ws[1])==4 and ws[1].isnumeric():
                                record['latest_date']=datetime(int(ws[1]),12,31).date()
                            elif len(ws[1])==2 and ws[1].isnumeric():
                                fch2=ws[0][0:2]+ws[1]
                                record['latest_date']=datetime(int(fch2),12,31).date()
    
                            
                else:
                    record[k]=vals
        if len(comms)>0:
            record['notes'] = comms
        if len(record)>1:
            records.append(record)
    return records

In [107]:
def import_records_from_workbook(filepath, workbook, worksheet, col_dictionary, create_record_function, **kwargs):
    wb = openpyxl.load_workbook(filepath / workbook, data_only=True)
    ws=wb[worksheet]
    row_count = ws.max_row+1
    records=list()
    for k in range(2,row_count):
        item=ws[k]
        record=create_record_function(item,col_dictionary,**kwargs)
        if record is not None:
            if type(record)==list:
                records.extend(record)
            elif type(record)==dict:
                records.append(record)
    return records


### Sth NSW RF

In [108]:
filename='SthnNSWRF_data_bionet2.xlsx'
worksheet='Fire'

In [109]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)

In [110]:
len(records)

11

In [111]:
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
11 rows updated
Database connection closed.


### Upland Basalt

In [113]:
filename='UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx'
wbindex[filename][worksheet][0][0:11]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3']

In [114]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)

In [115]:
len(records)

66

In [116]:
records[0:3]

[{'site_label': 'CRC09B7UVH',
  'fire_date': '2020-02-10',
  'earliest_date': datetime.date(2020, 2, 10),
  'latest_date': datetime.date(2020, 2, 10),
  'how_inferred': 'NPWS',
  'cause_of_ignition': 'lightning'},
 {'site_label': 'CRC09B7UVH',
  'fire_date': '2006-11-24',
  'earliest_date': datetime.date(2006, 11, 24),
  'latest_date': datetime.date(2006, 11, 24),
  'how_inferred': 'NPWS',
  'cause_of_ignition': 'Wildfire'},
 {'site_label': 'CRC09B7UVH',
  'fire_date': '1986',
  'earliest_date': datetime.date(1986, 1, 1),
  'latest_date': datetime.date(1986, 12, 31),
  'how_inferred': 'NPWS',
  'cause_of_ignition': 'Wildfire'}]

In [117]:
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
66 rows updated
Database connection closed.


### RF NE NSW SE Qld

In [118]:
filename='UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx'
wbindex[filename][worksheet][0][0:11]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3']

In [120]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
51 rows updated
Database connection closed.


### Yatteyattah


In [124]:
filename='UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx'
wbindex[filename][worksheet][0][0:11]

['Site',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m)']

In [125]:
col_dicts=[{'site_label':0,'fire_date':1,'how_inferred':4,'cause_of_ignition':7},
    {'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
20 rows updated
Database connection closed.


### Robertson Rainforest

In [131]:
filename='RobertsonRF_data_bionet2.xlsx'
wbindex[filename][worksheet][0][0:13]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m) min',
 'Scorch hgt (m) mas']

In [132]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
4 rows updated
Database connection closed.


### Alpine bogs

In [134]:
filename='UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx'
wbindex[filename][worksheet][0][0:13]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m)',
 '% Tree foliage scorch']

In [135]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
12 rows updated
Database connection closed.


### KNP Alpine Ash

In [137]:
filename='UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx'
wbindex[filename][worksheet][0][0:13]

['Site',
 'Replicate',
 'Date of last fire dd/mm/yyyy',
 'Date of penultimate fire',
 'Date of earlier fire',
 'How date inferred1',
 'How date inferred2',
 'How date inferred3',
 'Ignition cause1',
 'Ignition cause2',
 'Ignition cause3',
 'Scorch hgt (m) min',
 'Scorch hgt (m) max']

In [138]:
col_dicts=[{'site_label':0,'fire_date':2,'how_inferred':5,'cause_of_ignition':8},
    {'site_label':0,'fire_date':3,'how_inferred':6,'cause_of_ignition':9},
    {'site_label':0,'fire_date':4,'how_inferred':7,'cause_of_ignition':10}]
records = import_records_from_workbook(inputdir, filename, worksheet, col_dicts, create_record_function=create_fire_history_record)
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
8 rows updated
Database connection closed.


## Manual updates to the table

In [276]:
qries = ["""
UPDATE form.fire_history 
SET earliest_date='2020-01-01', latest_date='2020-01-31' 
WHERE fire_date IN ('__/01/2020', 'Jan 2020', 'Jan. 2020') 
AND earliest_date is NULL AND latest_date is NULL;
""",
"""
UPDATE form.fire_history 
SET earliest_date='2021-01-01', latest_date='2021-01-31' 
WHERE fire_date IN ('1/2021') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET earliest_date='1940-01-01', latest_date='1949-12-31' 
WHERE fire_date IN ('1940''s') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET earliest_date='1950-01-01', latest_date='1959-12-31' 
WHERE fire_date IN ('1950''s') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET earliest_date='2021-01-04', latest_date='2021-01-04' 
WHERE fire_date IN ('4/1/21') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET earliest_date='2019-11-09', latest_date='2019-11-09' 
WHERE fire_date IN ('9/11/2019') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET earliest_date='2019-11-01', latest_date='2019-11-30' 
WHERE fire_date IN ('Nov. 2019') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET earliest_date='2019-11-15', latest_date='2019-11-30' 
WHERE fire_date IN ('Late Nov. 2019') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET latest_date='1951-12-31' 
WHERE fire_date IN ('pre ~1951') 
AND earliest_date is NULL AND latest_date is NULL;
""", 
"""
UPDATE form.fire_history 
SET latest_date='1990-12-31' 
WHERE fire_date IN ('>30yrs') 
AND earliest_date is NULL AND latest_date is NULL;
"""]
 


In [277]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()

updated_rows=0
for upd in qries:
    cur.execute(upd)
    updated_rows = updated_rows + cur.rowcount

print(updated_rows)
conn.commit()
cur.close()
if conn is not None:
    conn.close()
    print('Database connection closed.')
 

Connecting to the PostgreSQL database...
0
Database connection closed.


### Newnes
Data from Newnes is present in each individual worksheet.
Edited site labels for BUD1 and BS1 to match data in database.

In [246]:
filename='Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm'
wbindex[filename].keys()
#worksheet='Fire'
#

dict_keys(['BS1', 'BS2', 'MW1', 'MW2', 'HV1', 'HV2', 'SS1', 'SS2', 'BUD1', 'BUD2', 'GGE1', 'GGE2', 'GGW1', 'GGW2', 'CW1', 'CW2', 'CC1', 'CC2', 'EW1', 'EW2', 'Species Metadata', 'Site', 'PivotFreq_Mar+Nov2020', 'Floristics', 'PivotCount-Mar+Nov2020', 'MaxCount-Mar+Nov2020_gt3', 'MaxCount-Mar+Nov2020_allspp', 'csvR_MaxCount_allspp', 'MaxFreq-Mar+Nov2020_gt3', 'Pivot-MarData', 'Sheet4', 'LiveResprouts_sum', 'Sheet5', 'FireKilled_sum', 'Sheet3', 'TotLive_freqscore', 'FireSurvival', 'Rfile-Ord', 'R-surv'])

In [270]:
wb = openpyxl.load_workbook(inputdir/filename, read_only=True)



In [271]:
def extract_fire_history(ws):
    records=list()
    for k in range(1,5):
        row = ws[k]
        for j in range(0,5):
            val=row[j].value
            if val is not None:
                if val.find("Sample Identifier")>=0:
                    target_label=ws.cell(row=k,column=j+3)
                if val.find("Fire History")>=0:
                    target_fh=ws.cell(row=k,column=j+1)
    #            print("%s/%s: %s" % (k,j,val))
    site_label=target_label.value.strip(" ")
    
    index=dict()
    switch={'Date':'fire_date','How inferred':'how_inferred','Cause of ignition':'cause_of_ignition'}
    k = target_fh.row
    for j in range(target_fh.column,target_fh.column+10):
        val=ws.cell(row=k,column=j).value
        if val in switch.keys():
            index[switch[val]]=j
    #print(index)

    for k in range(target_fh.row+1,target_fh.row+4):
        record={'site_label':site_label}
        comms=list()
        for w in index:
            j=index[w]
            val=ws.cell(row=k,column=j).value
            if val is not None:
                record[w]=val
                if w=='fire_date':
                    if isinstance(val,datetime):
                        record['fire_date']=str(val.date())
                        record['earliest_date']=val.date()
                        record['latest_date']=val.date()
                    elif isinstance(val,int):
                        record['fire_date']=str(val)
                        if val>0:
                            record['earliest_date']=datetime(val,1,1).date()
                            record['latest_date']=datetime(val,12,31).date()
                        else:
                            comms.append('Fire date is missing or empty')
                    elif vals.isnumeric():
                        record['fire_date']=str(val)
                        record['earliest_date']=datetime(int(val),1,1).date()
                        record['latest_date']=datetime(int(val),12,31).date()
                    else:
                        record['fire_date']=val
                        comms.append('Fire date given as: %s' % val)
                        found=re.findall("[<>]",val)
                        for i in found:
                            comms.append('max/min value given')
                            val=val.replace(i,"")
                        ws=val.split("-")
                        if len(ws)==2:
                            if len(ws[0])==4 and ws[0].isnumeric():
                                record['earliest_date']=datetime(int(ws[0]),1,1).date()
                            if len(ws[1])==4 and ws[1].isnumeric():
                                record['latest_date']=datetime(int(ws[1]),12,31).date()
                            elif len(ws[1])==2 and ws[1].isnumeric():
                                fch2=ws[0][0:2]+ws[1]
                                record['latest_date']=datetime(int(fch2),12,31).date()
    
        if len(comms)>0:
            record['notes'] = comms
        if len(record)>1:
            records.append(record) 
        return records
                

In [272]:
extract_fire_history(wb['BS1'])

[{'site_label': 'BS1',
  'fire_date': '2019-12-16',
  'earliest_date': datetime.date(2019, 12, 16),
  'latest_date': datetime.date(2019, 12, 16),
  'how_inferred': 'RFS records',
  'cause_of_ignition': 'lightning - Gospers Mtn'}]

In [273]:
records=list()
for ws in ('BS1', 'BS2', 'MW1', 'MW2', 'HV1', 'HV2', 'SS1', 'SS2', 'BUD1', 'BUD2', 'GGE1', 'GGE2', 'GGW1', 'GGW2', 'CW1', 'CW2', 'CC1', 'CC2', 'EW1', 'EW2'):
    records.extend(extract_fire_history(wb[ws]))

In [274]:
len(records)
##records

19

In [275]:
batch_upsert(params,"form.fire_history",records,keycol=('site_label','fire_date'), idx='fire_history_pkey1',execute=True)

Connecting to the PostgreSQL database...
19 rows updated
Database connection closed.


['Site',
 'Easting',
 'Northing',
 'Valley',
 'Elev',
 'Undermined',
 'Fire interval',
 'Census',
 'Date',
 'Scorch hgt',
 'Shb foliage scorch',
 "Shb foliage c'sume",
 'Herb foliage scorch',
 "Herb foliage c'sume",
 'Twig diam mean',
 'Twig diam se',
 'Peat depth burnt',
 'Peat extent burnt',
 'Peat fire index',
 'Postfire treehgt lower',
 'Postfire treehgt upper',
 'Postfire treehgt mode',
 'Postfire treecov',
 'Prefire shbhgt lower',
 'Prefire shbhgt upper',
 'Prefire shbhgt mode',
 'Prefire shbcov',
 'Postfire shbhgt lower',
 'Postfire shbhgt upper',
 'Postfire shbhgt mode',
 'Postfire shbcov',
 'Prefire hrbhgt lower',
 'Prefire hrbhgt upper',
 'Prefire hrbhgt mode',
 'Prefire hrbcov',
 'Postfire hrbhgt lower',
 'Postfire hrbhgt upper',
 'Postfire hrbhgt mode',
 'Postfire hrbcov',
 'Biomass A',
 'Biomass B',
 'Biomass C',
 'Biomass D',
 'Biomass E',
 'Mean dry (60C) biomass (g)',
 'CV biomass',
 'Mean biomass (g/m2)',
 'Native spp richness',
 'Sediment depth (mm) 1',
 'Sediment dep